# CIFAR-10

La première étape intermédiaire de notre projet est d'utiliser les algorithmes ci-dessous sur le célébre dataset CIFAR-10.

Les algorithmes à étudier :

**Modèles précédents**
- Modèle Linéaire
- Perceptron Multicouches

**Nouveaux modèles**
- ConvNet(s)
- ResNets / HighwayNets 
- RNN(s)

Pour chacun des algorithmes cités, il faut :
1. L'influence de tous les hyperparamètres des modèles
    - Structure
    - Fonctions d'activations
    - etc.
2. Les paramètres des algorithmes d'apprentissages
    - Learning Rate
    - Momentum
    - etc.
    
----

### Méthodologie
1. Créer un modèle classique.
2. Entraîner le modèle pendant 500 epochs.
3. Examiner sa courbe TensorBoard et son accuracy.
4. Augmenter un des hyperparamètre.
5. Réduire ce même hyperparamètre.
6. Dire l'influence de cette hyperparamètre sur la courbe et sur le modèle de manière générale.
7. Recommencer à partir de l'étape 3 pour tout les hyperparamètres possibles.

#### Hyperparamètres
- Batch size
- Learning rate
- Momentum
- Structure
- Fonction d'activation
- Initialization de kernel
- Regularizers (Dropout, L1 Norm, L2 Norm)

#### Qui s'occupe de quoi ?
- Perceptron (Mamadian)
- MLP (Réda M.)
- ConvNet (Reda B.)

In [21]:
import os
import numpy as np
from typing import List
from numpy.random import seed
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv2D, BatchNormalization, Input, Average, MaxPool2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.activations import relu, softmax, tanh
from tensorflow.keras.initializers import he_normal, glorot_uniform
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.random import set_seed

In [22]:
print("Version de TensorFlow :", tf.__version__)
print("Nom du GPU :", tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False)

Version de TensorFlow : 2.6.0
Nom du GPU : /device:GPU:0


2021-12-25 23:59:46.885120: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-25 23:59:46.885162: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Importation du dataset

In [23]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [24]:
NUM_CLASSES = 10
IMG_SIZE = x_train[0].shape

In [25]:
x_train = x_train.astype('float32') / 256
x_test = x_test.astype('float32') / 256

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [26]:
LOG_DIR = os.path.join("logs")
MODELS_DIR = os.path.join("models")

## Fixer les seeds

In [27]:
set_seed(42) # TensorFlow
seed(42) # NumPy

# Modèle linéaire

In [18]:
EPOCHS = 30
SHUFFLE = True

In [19]:
class Perceptron(Model):
    def __init__(self, activation_function, kernel_initializer, l2_val=0, l1_val=0):
        super(Perceptron, self).__init__()
        self._flatten_layer = Flatten()
        if l1_val > 0 and l2_val > 0:
            self._dense_layer = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2],
                                      activation=activation_function,
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=l1_l2(l1_val, l2_val),
                                      bias_regularizer=l1_l2(l1_val, l2_val)
                                      )
        elif l1_val > 0:
            self._dense_layer = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2],
                                      activation=activation_function,
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=l1(l1_val),
                                      bias_regularizer=l1(l1_val)
                                      )
        elif l2_val > 0:
            self._dense_layer = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2],
                                      activation=activation_function,
                                      kernel_initializer=kernel_initializer,
                                      kernel_regularizer=l2(l2_val),
                                      bias_regularizer=l2(l2_val)
                                      )
        else:
            self._dense_layer = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2],
                                      activation=activation_function,
                                      kernel_initializer=kernel_initializer
                                      )
        self._output_layer = Dense(NUM_CLASSES, activation=softmax)
                
    
    def call(self, inputs):
        x = self._flatten_layer(inputs)
        x = self._dense_layer(x)
        return self._output_layer(x)

In [20]:
EPOCHS = 500
BATCH_SIZE = 512

learning_rate = 0.01
momentum = 0.8
l2_val = 0.01
l1_val = 0.00
using_l1 = l1_val > 0
using_l2 = l2_val > 0
ki = glorot_uniform
fa = tanh
version = '_pft'

perceptron = Perceptron(tanh, glorot_uniform, l1_val=l1_val, l2_val=l2_val)
perceptron.compile(loss=categorical_crossentropy,
                   optimizer=SGD(learning_rate=learning_rate,
                                 momentum=momentum),
                   metrics=categorical_accuracy)

log_name = os.path.join(LOG_DIR,
                            "linear",
                            f"linear_model_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rate}_mo_{momentum}_{('l1_'+str(l1_val)+'_l2_'+str(l2_val) if using_l1 and using_l2 else ('l1_'+str(l1_val) if using_l1 else ('l2_'+str(l2_val) if using_l2 else '')))}_ki_{ki.__name__}_af_{fa.__name__}{version}")

print(log_name)
perceptron.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test, y_test), shuffle=True, callbacks=[TensorBoard(log_name, histogram_freq=1)])

logs/linear/linear_model_ep_500_bs_512_opt_SGD_lr_0.01_mo_0.8_l2_0.01_ki_GlorotUniform_af_tanh_pft


2021-12-20 19:21:45.605928: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-20 19:21:45.605952: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-20 19:21:45.606607: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/500
 3/98 [..............................] - ETA: 3s - loss: 33.0334 - categorical_accuracy: 0.1165 

2021-12-20 19:21:46.337840: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-20 19:21:46.425094: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-20 19:21:46.425103: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-20 19:21:46.467851: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-20 19:21:46.468142: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-20 19:21:46.468613: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/linear/linear_model_ep_500_bs_512_opt_SGD_lr_0.01_mo_0.8_l2_0.01_ki_GlorotUniform_af_tanh_pft/train/plugins/profile/2021_12_20_19_21_46

2021-12-20 19:21:46.469041: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/line

97/98 [============================>.] - ETA: 0s - loss: 30.0985 - categorical_accuracy: 0.3104

2021-12-20 19:21:49.426180: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


98/98 [==============================] - 3s 34ms/step - loss: 30.0804 - categorical_accuracy: 0.3102 - val_loss: 27.2481 - val_categorical_accuracy: 0.3747
Epoch 2/500
98/98 [==============================] - 3s 31ms/step - loss: 24.9466 - categorical_accuracy: 0.3644 - val_loss: 22.7203 - val_categorical_accuracy: 0.3496
Epoch 3/500
98/98 [==============================] - 3s 31ms/step - loss: 20.7820 - categorical_accuracy: 0.3809 - val_loss: 18.9344 - val_categorical_accuracy: 0.3977
Epoch 4/500
98/98 [==============================] - 3s 31ms/step - loss: 17.3872 - categorical_accuracy: 0.3839 - val_loss: 15.8946 - val_categorical_accuracy: 0.3808
Epoch 5/500
59/98 [=================>............] - ETA: 1s - loss: 15.0841 - categorical_accuracy: 0.3909

KeyboardInterrupt: 

## Trouver le meilleur modèle

In [9]:
def linear_model(activation_function, kernel_initializer, learning_rate, momentum, batch_size, version=''):
    input_layer = Input(shape=(32, 32, 3))

    hidden_layers = Flatten()(input_layer)
    hidden_layers = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2], activation=activation_function, kernel_initializer=kernel_initializer)(hidden_layers)

    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    linear_model = Model(input_layer, output_layer)
    
    linear_model.compile(loss=categorical_crossentropy,
                         optimizer=SGD(learning_rate=learning_rate,
                                       momentum=momentum),
                         metrics=categorical_accuracy)
    log_name = os.path.join(LOG_DIR,
                            "linear",
                            f"linear_model_ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{learning_rate}_mo_{momentum}_ki_{kernel_initializer.__name__}_af_{activation_function.__name__}{version}")
    history = linear_model.fit(x_train,
                               y_train,
                               batch_size=batch_size,
                               epochs=EPOCHS,
                               validation_data=(x_test, y_test),
                               shuffle=SHUFFLE,
                               callbacks=[TensorBoard(log_name, histogram_freq=1)])
    return linear_model, history

Les hyperparamètres que nous souhaitons ajuster :

In [10]:
learning_rates = [0.01, 0.05, 0.1, 0.2]
kernel_initializers = [glorot_uniform, he_normal]
activation_functions = [tanh, relu]
batch_sizes = [64, 128, 256]
momentums = [0, 0.5, 0.9]

In [ ]:
# Grid Search V1

best_model_accuracy = 0
counter = 0
linear_models = {}

for mo in momentums:
    for bs in batch_sizes:
        for af in activation_functions:
            for ki in kernel_initializers:
                for lr in learning_rates:
                    print(f"Iteration n°{counter} | af: {af.__name__} - ki: {ki.__name__} - SGD lr: {lr} / mo: {mo} - bs: {bs}")
                    model_name = f"af_{af.__name__}_ki_{ki.__name__}_sgd_lr_{lr}_mo_{mo}_bs_{bs}"
                    model, history = linear_model(af, ki, lr, mo, bs)
                    accuracy = round(history.history["categorical_accuracy"][-1], 3)
                    print(f"Accuracy : {accuracy}")
                    linear_models[model_name] = accuracy
                    if accuracy > best_model_accuracy:
                        model.save(f"models/linear/{str(accuracy)+'_'+model_name}.keras")
                        best_model_accuracy = accuracy
                    counter += 1

Après une trentaine d'épochs, nous avons remarqué que nous avons les pires résultats lorsque nous utilisons un learning rate égale à 0.1 et 0.2. Du coup, nous allons les supprimer lors de la prochaine version. Nous allons aussi en profiter pour accrocher les fonctions d'activations à l'initialisateur de kernel conseillé durant le cours (tanh -> glorot_uniform et relu -> he_normal)

**Résultat:**

Les meilleurs modèles que nous avons ont en commun un batch size de 64, un learning rate de 0.05, peu de momentum et enfin le combo relu/he_normal. 

In [12]:
#best_models_sorted = sorted(linear_models.items(), key=lambda x: x[1], reverse=True)
#print(best_models_sorted[:5])

In [13]:
best_linear_model = load_model('models/linear/af_relu_ki_HeNormal_sgd_lr_0.05_mo_0_bs_64_0.708.keras')

2021-12-17 10:37:14.580906: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-17 10:37:14.580954: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [14]:
test_loss, test_accuracy = best_linear_model.evaluate(x_test, y_test)
print(f"Test loss : {round(test_loss, 2)}")
print(f"Test accuracy : {round(test_accuracy, 2) * 100}%")

2021-12-17 10:37:17.547261: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-17 10:37:17.550501: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-17 10:37:17.604341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 4ms/step - loss: 1.5417 - categorical_accuracy: 0.5075
Test loss : 1.54
Test accuracy : 51.0%


Pour l'instant, notre meilleur modèle prédit correctement 51% du temps, ce qui est pas mal sur 10 catégories.
Nous allons donc continuer de chercher le meilleure modèle en restant proche de ces paramètres, nous allons également ajouter à nos hyperparamètres, une forme de régularisation (norme l1, l2).

In [28]:
EPOCHS = 500

In [29]:
learning_rates = [0.04, 0.05, 0.06]
momentums = [0, 0.10, 0.25]
batch_size = 512
kernel_initializer = he_normal
activation_function = relu

In [30]:
# Panda fine-tuning
learning_rate = 0.035
momentum = 0.5
l2_val = 0.01


input_layer = Input(shape=(32, 32, 3))

hidden_layers = Flatten()(input_layer)
hidden_layers = Dense(IMG_SIZE[0] * IMG_SIZE[1] * IMG_SIZE[2], activation=activation_function, kernel_initializer=kernel_initializer)(hidden_layers)

output_layer = Dense(NUM_CLASSES, activation=softmax,
                     kernel_regularizer = l2(l2_val),
                     bias_regularizer = l2(l2_val))(hidden_layers)
linear_model = Model(input_layer, output_layer)

linear_model.compile(loss=categorical_crossentropy,
                     optimizer=SGD(learning_rate=learning_rate,
                                   momentum=momentum),
                     metrics=categorical_accuracy)
log_name = os.path.join(LOG_DIR,
                        "linear",
                        f"linear_model_ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{learning_rate}_mo_{momentum}_ki_{kernel_initializer.__name__}_af_{activation_function.__name__}_l2_{l2_val}_pft")
history = linear_model.fit(x_train,
                           y_train,
                           batch_size=batch_size,
                           epochs=EPOCHS,
                           validation_data=(x_test, y_test),
                           shuffle=SHUFFLE,
                           callbacks=[TensorBoard(log_name, histogram_freq=1)])
accuracy = round(history.history["val_categorical_accuracy"][-1], 3)
print(f"Accuracy : {accuracy}")

2021-12-17 11:08:07.889242: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-17 11:08:07.889262: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-17 11:08:07.889370: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/500
 3/98 [..............................] - ETA: 3s - loss: 3.4384 - categorical_accuracy: 0.1009 

2021-12-17 11:08:08.537147: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-17 11:08:08.612097: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-17 11:08:08.612106: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-17 11:08:08.649351: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-17 11:08:08.649644: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-17 11:08:08.650689: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/linear/linear_model_ep_500_bs_512_opt_SGD_lr_0.035_mo_0.5_ki_HeNormal_af_relu_l2_0.01_pft/train/plugins/profile/2021_12_17_11_08_08

2021-12-17 11:08:08.651127: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/linear/l

97/98 [============================>.] - ETA: 0s - loss: 2.2165 - categorical_accuracy: 0.3292

2021-12-17 11:08:10.966469: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

Epoch 295/500
98/98 [==============================] - 2s 24ms/step - loss: 1.0344 - categorical_accuracy: 0.7277 - val_loss: 1.4556 - val_categorical_accuracy: 0.5443
Epoch 296/500
98/98 [==============================] - 2s 23ms/step - loss: 1.0316 - categorical_accuracy: 0.7304 - val_loss: 1.4200 - val_categorical_accuracy: 0.5587
Epoch 297/500
98/98 [==============================] - 2s 25ms/step - loss: 1.0303 - categorical_accuracy: 0.7301 - val_loss: 1.4108 - val_categorical_accuracy: 0.5625
Epoch 298/500
98/98 [==============================] - 2s 25ms/step - loss: 1.0316 - categorical_accuracy: 0.7282 - val_loss: 1.4282 - val_categorical_accuracy: 0.5581
Epoch 299/500
98/98 [==============================] - 2s 24ms/step - loss: 1.0279 - categorical_accuracy: 0.7322 - val_loss: 1.4071 - val_categorical_accuracy: 0.5638
Epoch 300/500
98/98 [==============================] - 2s 24ms/step - loss: 1.0284 - categorical_accuracy: 0.7296 - val_loss: 1.4291 - val_categorical_accuracy:

Epoch 344/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9900 - categorical_accuracy: 0.7489 - val_loss: 1.4146 - val_categorical_accuracy: 0.5645
Epoch 345/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9902 - categorical_accuracy: 0.7477 - val_loss: 1.4234 - val_categorical_accuracy: 0.5610
Epoch 346/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9897 - categorical_accuracy: 0.7487 - val_loss: 1.4173 - val_categorical_accuracy: 0.5637
Epoch 347/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9888 - categorical_accuracy: 0.7484 - val_loss: 1.4234 - val_categorical_accuracy: 0.5595
Epoch 348/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9890 - categorical_accuracy: 0.7479 - val_loss: 1.4259 - val_categorical_accuracy: 0.5612
Epoch 349/500
98/98 [==============================] - 2s 23ms/step - loss: 0.9851 - categorical_accuracy: 0.7519 - val_loss: 1.4447 - val_categorical_accuracy:

Epoch 393/500
98/98 [==============================] - 2s 25ms/step - loss: 0.9522 - categorical_accuracy: 0.7666 - val_loss: 1.4082 - val_categorical_accuracy: 0.5697
Epoch 394/500
98/98 [==============================] - 3s 27ms/step - loss: 0.9521 - categorical_accuracy: 0.7659 - val_loss: 1.4119 - val_categorical_accuracy: 0.5683
Epoch 395/500
98/98 [==============================] - 2s 25ms/step - loss: 0.9532 - categorical_accuracy: 0.7668 - val_loss: 1.4186 - val_categorical_accuracy: 0.5644
Epoch 396/500
98/98 [==============================] - 3s 26ms/step - loss: 0.9500 - categorical_accuracy: 0.7691 - val_loss: 1.4271 - val_categorical_accuracy: 0.5613
Epoch 397/500
98/98 [==============================] - 3s 26ms/step - loss: 0.9480 - categorical_accuracy: 0.7690 - val_loss: 1.4314 - val_categorical_accuracy: 0.5602
Epoch 398/500
98/98 [==============================] - 3s 27ms/step - loss: 0.9508 - categorical_accuracy: 0.7673 - val_loss: 1.4366 - val_categorical_accuracy:

Epoch 442/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9177 - categorical_accuracy: 0.7832 - val_loss: 1.4219 - val_categorical_accuracy: 0.5655
Epoch 443/500
98/98 [==============================] - 2s 24ms/step - loss: 0.9172 - categorical_accuracy: 0.7836 - val_loss: 1.4566 - val_categorical_accuracy: 0.5556
Epoch 444/500
98/98 [==============================] - 3s 26ms/step - loss: 0.9162 - categorical_accuracy: 0.7846 - val_loss: 1.4287 - val_categorical_accuracy: 0.5646
Epoch 445/500
98/98 [==============================] - 2s 25ms/step - loss: 0.9133 - categorical_accuracy: 0.7872 - val_loss: 1.4328 - val_categorical_accuracy: 0.5603
Epoch 446/500
98/98 [==============================] - 3s 26ms/step - loss: 0.9119 - categorical_accuracy: 0.7873 - val_loss: 1.4312 - val_categorical_accuracy: 0.5676
Epoch 447/500
98/98 [==============================] - 3s 27ms/step - loss: 0.9128 - categorical_accuracy: 0.7871 - val_loss: 1.4305 - val_categorical_accuracy:

Epoch 491/500
98/98 [==============================] - 2s 23ms/step - loss: 0.8861 - categorical_accuracy: 0.7983 - val_loss: 1.4441 - val_categorical_accuracy: 0.5626
Epoch 492/500
98/98 [==============================] - 2s 23ms/step - loss: 0.8849 - categorical_accuracy: 0.7986 - val_loss: 1.4338 - val_categorical_accuracy: 0.5683
Epoch 493/500
98/98 [==============================] - 2s 23ms/step - loss: 0.8861 - categorical_accuracy: 0.7990 - val_loss: 1.4424 - val_categorical_accuracy: 0.5655
Epoch 494/500
98/98 [==============================] - 2s 23ms/step - loss: 0.8836 - categorical_accuracy: 0.8001 - val_loss: 1.4283 - val_categorical_accuracy: 0.5710
Epoch 495/500
98/98 [==============================] - 2s 25ms/step - loss: 0.8832 - categorical_accuracy: 0.7987 - val_loss: 1.4280 - val_categorical_accuracy: 0.5729
Epoch 496/500
98/98 [==============================] - 2s 24ms/step - loss: 0.8851 - categorical_accuracy: 0.7985 - val_loss: 1.4426 - val_categorical_accuracy:

In [ ]:
#model_name = f"af_{activation_function.__name__}_ki_{kernel_initializer.__name__}_sgd_lr_{0.05}_mo_{0.10}_bs_{batch_size}"
#linear_model.save(f"models/linear/{str(accuracy)+'_'+model_name}.keras")

In [163]:
# Grid Search V2

best_model_accuracy = 0
counter = 0
linear_models = {}

for lr in learning_rates:
    for mo in momentums:
        counter += 1
        print(f"Iteration n°{counter} | af: {activation_function.__name__} - ki: {kernel_initializer.__name__} - SGD lr: {lr} / mo: {mo} - bs: {batch_size}")
        model_name = f"af_{activation_function.__name__}_ki_{kernel_initializer.__name__}_sgd_lr_{lr}_mo_{mo}_bs_{batch_size}"
        model, history = linear_model(activation_function, kernel_initializer, lr, mo, batch_size, version='_v2')
        accuracy = round(history.history["val_categorical_accuracy"][-1], 3)
        print(f"Accuracy : {accuracy}")
        linear_models[model_name] = accuracy
        if accuracy > best_model_accuracy:
            model.save(f"models/linear/{str(accuracy)+'_'+model_name}.keras")
            best_model_accuracy = accuracy

Iteration n°1 | af: relu - ki: HeNormal - SGD lr: 0.04 / mo: 0 - bs: 64


2021-12-17 00:17:58.827727: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-17 00:17:58.827742: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-17 00:17:58.827824: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/50
  7/782 [..............................] - ETA: 14s - loss: 5.4403 - categorical_accuracy: 0.0960

2021-12-17 00:17:59.926755: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-17 00:18:00.002039: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-17 00:18:00.002052: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-17 00:18:00.029046: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-17 00:18:00.029373: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-17 00:18:00.030466: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/linear/linear_model_ep_50_bs_64_opt_SGD_lr_0.04_mo_0_ki_HeNormal_af_relu_v2/train/plugins/profile/2021_12_17_00_18_00

2021-12-17 00:18:00.030903: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/linear/linear_model_ep

779/782 [============================>.] - ETA: 0s - loss: 1.8277 - categorical_accuracy: 0.3605

...

2021-12-17 00:26:33.724586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-17 00:26:33.810521: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-17 00:26:33.810529: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-17 00:26:33.836342: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-17 00:26:33.836642: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-17 00:26:33.837645: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/linear/linear_model_ep_50_bs_64_opt_SGD_lr_0.04_mo_0.1_ki_HeNormal_af_relu_v2/train/plugins/profile/2021_12_17_00_26_33

2021-12-17 00:26:33.838088: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/linear/linear_model_

780/782 [============================>.] - ETA: 0s - loss: 1.8155 - categorical_accuracy: 0.3623

2021-12-17 00:26:45.032421: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 12s 15ms/step - loss: 1.8155 - categorical_accuracy: 0.3623 - val_loss: 1.7611 - val_categorical_accuracy: 0.3611
Epoch 2/50
782/782 [==============================] - 12s 15ms/step - loss: 1.6165 - categorical_accuracy: 0.4278 - val_loss: 1.7516 - val_categorical_accuracy: 0.3535
Epoch 3/50
782/782 [==============================] - 12s 15ms/step - loss: 1.5368 - categorical_accuracy: 0.4601 - val_loss: 1.5994 - val_categorical_accuracy: 0.4249
Epoch 4/50
782/782 [==============================] - 11s 15ms/step - loss: 1.4816 - categorical_accuracy: 0.4804 - val_loss: 1.5883 - val_categorical_accuracy: 0.4224
Epoch 5/50
782/782 [==============================] - 11s 14ms/step - loss: 1.4353 - categorical_accuracy: 0.4980 - val_loss: 1.6067 - val_categorical_accuracy: 0.4298
Epoch 6/50
782/782 [==============================] - 11s 14ms/step - loss: 1.3964 - categorical_accuracy: 0.5096 - val_loss: 1.7055 - val_categorical_accuracy: 0.4037
Epo

KeyboardInterrupt: 

---

# MLP

Pour voir l'influence des hyperparamètres sur un MLP, nous avons d'abord besoin de fixer le nombre d'époques et le nombre de layer sur notre couche de sortie.

In [15]:
EPOCHS = 500
SHUFFLE = True
NUM_CLASSES = 10

Pour nous faciliter la tâche, nous allons créer un modèle principal appelé *main* puis nous allons ajouter, modifier ou enlever un hyperparamètre à la fois pour que l'on puisse aperçevoir son influence sur le modèle. Pour chacun des hyperparamètres déjà présent dans notre modèle principal, nous allons prendre une valeur excessivement élevè et une valeur très basse. En revanche, pour les hyperparamètres qui n'y s'y trouvent pas (ex: Dropout, L2), nous allons les ajouter à notre modèle et comparer.

## Modèle main

Le modèle `main` est notre modèle principale, ce sera celui avec lequelle nous comparerons tout les autres prochains modèles et essayer de distinguer le plus clairement possible l'influence des hyperparamètres que nous modifierons. Nous avons choisi les hyperparamètres par défaut lorsque c'était possible et avons choisi le reste selon notre intuition.

In [6]:
def MLP_main(num_layer: int, nodes_by_layers: List[int]) -> Model:
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = Flatten()(input_layer)
    
    for n in range(num_layer):
        hidden_layers = Dense(nodes_by_layers[n], activation=relu, kernel_initializer=he_normal)(hidden_layers)
        
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)

Voici les hyperparamètres de notre modèle :

In [7]:
batch_size = 256
num_layer = 6
nodes_by_layers = [32, 16, 64, 32, 16, 64]
learning_rate = 0.01
momentum = 0.45

Tout au long de notre section sur le MLP, nous n'utiliserons comme optimiseur uniquement Stochastic Gradient Descent (SGD). Nous pouvons désormais instancier et entraîner notre modèle.

In [11]:
mlp = MLP_main(num_layer, nodes_by_layers)

mlp.compile(
    loss=categorical_crossentropy,
    optimizer=SGD(learning_rate=learning_rate, momentum=momentum),
    metrics=categorical_accuracy
           )

MLP_LOG = os.path.join(LOG_DIR, "mlp",
                f"ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{lr}_mom_{mom}_af_relu_layers_{'_'.join(str(e) for e in nodes_by_layers)}_main")

MLP_MODELS = os.path.join(MODELS_DIR, "mlp",
                f"ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{lr}_mom_{mom}_af_relu_layers_{'_'.join(str(e) for e in nodes_by_layers)}_main.keras")

mlp.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=EPOCHS,
        validation_data=(x_test, y_test),
        shuffle=SHUFFLE,
        callbacks=[TensorBoard(MLP_LOG)]
       )
mlp.save(MLP_MODELS)

2021-12-25 12:27:19.861555: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-25 12:27:19.861604: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-12-25 12:27:19.937281: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-25 12:27:19.937294: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-25 12:27:19.937327: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-25 12:27:20.733856: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (register

Epoch 1/500
 20/196 [==>...........................] - ETA: 1s - loss: 2.3048 - categorical_accuracy: 0.1227

2021-12-25 12:27:21.052979: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-25 12:27:21.052988: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-25 12:27:21.062513: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-25 12:27:21.064246: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-25 12:27:21.067945: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/mlp/ep_500_bs_256_opt_SGD_lr_0.01_mom_0.45_af_relu_layers_32_16_64_32_16_64_main/train/plugins/profile/2021_12_25_12_27_21

2021-12-25 12:27:21.068594: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/mlp/ep_500_bs_256_opt_SGD_lr_0.01_mom_0.45_af_relu_layers_32_16_64_32_16_64_main/train/plugins/profile/2021_12_25_12_27_21/MacBook-Pro-de-Reda.local.trace.json.gz
2021-12-2

193/196 [============================>.] - ETA: 0s - loss: 2.1645 - categorical_accuracy: 0.1994

2021-12-25 12:27:22.626390: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

Epoch 99/500
196/196 [==============================] - 2s 8ms/step - loss: 1.2854 - categorical_accuracy: 0.5366 - val_loss: 1.4587 - val_categorical_accuracy: 0.4821
Epoch 100/500
196/196 [==============================] - 2s 8ms/step - loss: 1.2798 - categorical_accuracy: 0.5372 - val_loss: 1.5161 - val_categorical_accuracy: 0.4565
Epoch 101/500
196/196 [==============================] - 2s 8ms/step - loss: 1.2803 - categorical_accuracy: 0.5349 - val_loss: 1.4692 - val_categorical_accuracy: 0.4781
Epoch 102/500
196/196 [==============================] - 2s 8ms/step - loss: 1.2817 - categorical_accuracy: 0.5370 - val_loss: 1.4985 - val_categorical_accuracy: 0.4666
Epoch 103/500
196/196 [==============================] - 2s 8ms/step - loss: 1.2681 - categorical_accuracy: 0.5431 - val_loss: 1.5623 - val_categorical_accuracy: 0.4554
Epoch 104/500
196/196 [==============================] - 2s 8ms/step - loss: 1.2793 - categorical_accuracy: 0.5385 - val_loss: 1.5797 - val_categorical_accu

196/196 [==============================] - 2s 8ms/step - loss: 1.1801 - categorical_accuracy: 0.5779 - val_loss: 1.6351 - val_categorical_accuracy: 0.4493
Epoch 196/500
196/196 [==============================] - 2s 9ms/step - loss: 1.1785 - categorical_accuracy: 0.5758 - val_loss: 1.8117 - val_categorical_accuracy: 0.4259
Epoch 197/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1876 - categorical_accuracy: 0.5750 - val_loss: 1.6541 - val_categorical_accuracy: 0.4360
Epoch 198/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1794 - categorical_accuracy: 0.5762 - val_loss: 1.6444 - val_categorical_accuracy: 0.4434
Epoch 199/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1798 - categorical_accuracy: 0.5745 - val_loss: 1.5439 - val_categorical_accuracy: 0.4709
Epoch 200/500
196/196 [==============================] - 2s 9ms/step - loss: 1.1746 - categorical_accuracy: 0.5775 - val_loss: 1.5571 - val_categorical_accuracy: 0.4681


196/196 [==============================] - 2s 8ms/step - loss: 1.1055 - categorical_accuracy: 0.6031 - val_loss: 1.6142 - val_categorical_accuracy: 0.4607
Epoch 292/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1060 - categorical_accuracy: 0.6028 - val_loss: 1.6149 - val_categorical_accuracy: 0.4586
Epoch 293/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1099 - categorical_accuracy: 0.6013 - val_loss: 1.6760 - val_categorical_accuracy: 0.4546
Epoch 294/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1097 - categorical_accuracy: 0.6010 - val_loss: 1.5178 - val_categorical_accuracy: 0.4804
Epoch 295/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1103 - categorical_accuracy: 0.6009 - val_loss: 1.5809 - val_categorical_accuracy: 0.4704
Epoch 296/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1102 - categorical_accuracy: 0.6022 - val_loss: 1.5404 - val_categorical_accuracy: 0.4765


196/196 [==============================] - 2s 8ms/step - loss: 1.0577 - categorical_accuracy: 0.6207 - val_loss: 1.6824 - val_categorical_accuracy: 0.4637
Epoch 388/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0616 - categorical_accuracy: 0.6184 - val_loss: 1.6386 - val_categorical_accuracy: 0.4698
Epoch 389/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0540 - categorical_accuracy: 0.6213 - val_loss: 1.6546 - val_categorical_accuracy: 0.4633
Epoch 390/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0596 - categorical_accuracy: 0.6201 - val_loss: 1.6087 - val_categorical_accuracy: 0.4703
Epoch 391/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0617 - categorical_accuracy: 0.6190 - val_loss: 1.6666 - val_categorical_accuracy: 0.4677
Epoch 392/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0502 - categorical_accuracy: 0.6231 - val_loss: 1.7537 - val_categorical_accuracy: 0.4436


196/196 [==============================] - 2s 8ms/step - loss: 1.0149 - categorical_accuracy: 0.6340 - val_loss: 1.7568 - val_categorical_accuracy: 0.4516
Epoch 484/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0246 - categorical_accuracy: 0.6321 - val_loss: 1.7128 - val_categorical_accuracy: 0.4652
Epoch 485/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0158 - categorical_accuracy: 0.6339 - val_loss: 1.6854 - val_categorical_accuracy: 0.4660
Epoch 486/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0242 - categorical_accuracy: 0.6306 - val_loss: 1.6429 - val_categorical_accuracy: 0.4756
Epoch 487/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0180 - categorical_accuracy: 0.6338 - val_loss: 1.7081 - val_categorical_accuracy: 0.4486
Epoch 488/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0165 - categorical_accuracy: 0.6345 - val_loss: 1.8166 - val_categorical_accuracy: 0.4414


Regardons la courbe de loss et d'accuracy de notre modèle principal. 

En bleu, les courbes sur les données de validation. Et en orange, les courbe sur les données d'entraînement.

![Loss_main](images/MLP_main_loss.png)

Nous pouvons voir que notre modèle stagne sur nos données de validation à partir de 50 époques, ce qui signifie qu'il sur-apprend après ces 50 époques. La courbe de loss nous permettra d'en être définitivement sûr.

![Accuracy](images/MLP_main_accuracy.png)


On peut désormais en être sûr, notre courbe de loss nous apprend que notre modèle **sur-apprend** mais après environ 150 époques. On peut le voir car notre courbe bleu (données de validation) augmente de plus en plus.

## Nombre de couches

Nous allons changer le nombre de couches de notre modèle, notre modèle initial contient 6 couches. Nous allons comparer avec un modèle à 2 puis à 20 couches pour voir l'importance de cette hyperparamètre.

In [9]:
batch_size = 256
learning_rate = 0.01
momentum = 0.45
num_layers = [2, 20]
nodes_by_layers = [32, 16, 64, 32, 16, 64, 32, 16, 64, 32, 16, 64, 32, 16, 64, 32, 16, 64, 32, 16]

In [12]:
for i, num_layer in enumerate(num_layers):
    mlp = MLP_main(num_layer, nodes_by_layers)

    mlp.compile(
        loss=categorical_crossentropy,
        optimizer=SGD(learning_rate=learning_rate, momentum=momentum),
        metrics=categorical_accuracy
               )

    MLP_LOG = os.path.join(LOG_DIR, "mlp",
                    f"ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{learning_rate}_mom_{momentum}_af_relu_layers_{'_'.join(str(e) for e in nodes_by_layers)}_a_{i+1}")

    MLP_MODELS = os.path.join(MODELS_DIR, "mlp",
                    f"ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{learning_rate}_mom_{momentum}_af_relu_layers_{'_'.join(str(e) for e in nodes_by_layers)}_a_{i+1}.keras")

    mlp.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[TensorBoard(MLP_LOG)]
           )
    mlp.save(MLP_MODELS)

2021-12-25 12:41:01.152753: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-25 12:41:01.152764: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-25 12:41:01.152777: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/500
 14/196 [=>............................] - ETA: 1s - loss: 2.2900 - categorical_accuracy: 0.1177

2021-12-25 12:41:01.809196: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-25 12:41:01.858128: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-25 12:41:01.858138: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-25 12:41:01.876342: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-25 12:41:01.876621: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-25 12:41:01.877665: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/mlp/ep_500_bs_256_opt_SGD_lr_0.01_mom_0.45_af_relu_layers_32_16_64_32_16_64_32_16_64_32_16_64_32_16_64_32_16_64_32_16_a_1/train/plugins/profile/2021_12_25_12_41_01

2021-12-25 12:41:01.878117: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data fo

190/196 [============================>.] - ETA: 0s - loss: 2.1597 - categorical_accuracy: 0.2019

2021-12-25 12:41:03.163999: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

196/196 [==============================] - 3s 14ms/step - loss: 1.4343 - categorical_accuracy: 0.4811 - val_loss: 1.5748 - val_categorical_accuracy: 0.4288
Epoch 98/500
196/196 [==============================] - 3s 14ms/step - loss: 1.4292 - categorical_accuracy: 0.4831 - val_loss: 1.9694 - val_categorical_accuracy: 0.3604
Epoch 99/500
196/196 [==============================] - 3s 14ms/step - loss: 1.4303 - categorical_accuracy: 0.4817 - val_loss: 1.6289 - val_categorical_accuracy: 0.4173
Epoch 100/500
196/196 [==============================] - 3s 13ms/step - loss: 1.4324 - categorical_accuracy: 0.4815 - val_loss: 1.6311 - val_categorical_accuracy: 0.4111
Epoch 101/500
196/196 [==============================] - 3s 13ms/step - loss: 1.4199 - categorical_accuracy: 0.4853 - val_loss: 1.6056 - val_categorical_accuracy: 0.4294
Epoch 102/500
196/196 [==============================] - 3s 13ms/step - loss: 1.4276 - categorical_accuracy: 0.4856 - val_loss: 1.5697 - val_categorical_accuracy: 0.4

196/196 [==============================] - 3s 13ms/step - loss: 1.2965 - categorical_accuracy: 0.5339 - val_loss: 1.6075 - val_categorical_accuracy: 0.4488
Epoch 194/500
196/196 [==============================] - 3s 13ms/step - loss: 1.2966 - categorical_accuracy: 0.5353 - val_loss: 1.6253 - val_categorical_accuracy: 0.4408
Epoch 195/500
196/196 [==============================] - 3s 13ms/step - loss: 1.2983 - categorical_accuracy: 0.5329 - val_loss: 1.7142 - val_categorical_accuracy: 0.4228
Epoch 196/500
196/196 [==============================] - 3s 13ms/step - loss: 1.2918 - categorical_accuracy: 0.5373 - val_loss: 1.5949 - val_categorical_accuracy: 0.4489
Epoch 197/500
196/196 [==============================] - 3s 13ms/step - loss: 1.2927 - categorical_accuracy: 0.5355 - val_loss: 1.5636 - val_categorical_accuracy: 0.4534
Epoch 198/500
196/196 [==============================] - 3s 13ms/step - loss: 1.2851 - categorical_accuracy: 0.5375 - val_loss: 1.5875 - val_categorical_accuracy: 0

196/196 [==============================] - 3s 13ms/step - loss: 1.2070 - categorical_accuracy: 0.5685 - val_loss: 1.7004 - val_categorical_accuracy: 0.4392
Epoch 290/500
196/196 [==============================] - 3s 13ms/step - loss: 1.2069 - categorical_accuracy: 0.5683 - val_loss: 1.5853 - val_categorical_accuracy: 0.4668
Epoch 291/500
196/196 [==============================] - 3s 13ms/step - loss: 1.2106 - categorical_accuracy: 0.5672 - val_loss: 1.6512 - val_categorical_accuracy: 0.4517
Epoch 292/500
196/196 [==============================] - 3s 13ms/step - loss: 1.2049 - categorical_accuracy: 0.5681 - val_loss: 1.6741 - val_categorical_accuracy: 0.4510
Epoch 293/500
196/196 [==============================] - 3s 13ms/step - loss: 1.2098 - categorical_accuracy: 0.5667 - val_loss: 1.5958 - val_categorical_accuracy: 0.4622
Epoch 294/500
196/196 [==============================] - 3s 13ms/step - loss: 1.2069 - categorical_accuracy: 0.5681 - val_loss: 1.5693 - val_categorical_accuracy: 0

196/196 [==============================] - 3s 13ms/step - loss: 1.1469 - categorical_accuracy: 0.5883 - val_loss: 1.6274 - val_categorical_accuracy: 0.4615
Epoch 386/500
196/196 [==============================] - 3s 13ms/step - loss: 1.1474 - categorical_accuracy: 0.5895 - val_loss: 1.6073 - val_categorical_accuracy: 0.4629
Epoch 387/500
196/196 [==============================] - 3s 13ms/step - loss: 1.1490 - categorical_accuracy: 0.5921 - val_loss: 1.6151 - val_categorical_accuracy: 0.4685
Epoch 388/500
196/196 [==============================] - 3s 13ms/step - loss: 1.1507 - categorical_accuracy: 0.5886 - val_loss: 1.6520 - val_categorical_accuracy: 0.4533
Epoch 389/500
196/196 [==============================] - 3s 13ms/step - loss: 1.1485 - categorical_accuracy: 0.5893 - val_loss: 1.6152 - val_categorical_accuracy: 0.4626
Epoch 390/500
196/196 [==============================] - 3s 13ms/step - loss: 1.1493 - categorical_accuracy: 0.5898 - val_loss: 1.6115 - val_categorical_accuracy: 0

196/196 [==============================] - 3s 13ms/step - loss: 1.1062 - categorical_accuracy: 0.6075 - val_loss: 1.7491 - val_categorical_accuracy: 0.4370
Epoch 482/500
196/196 [==============================] - 3s 13ms/step - loss: 1.1073 - categorical_accuracy: 0.6063 - val_loss: 1.6577 - val_categorical_accuracy: 0.4494
Epoch 483/500
196/196 [==============================] - 3s 13ms/step - loss: 1.1068 - categorical_accuracy: 0.6051 - val_loss: 1.6700 - val_categorical_accuracy: 0.4542
Epoch 484/500
196/196 [==============================] - 3s 13ms/step - loss: 1.1039 - categorical_accuracy: 0.6078 - val_loss: 1.6561 - val_categorical_accuracy: 0.4622
Epoch 485/500
196/196 [==============================] - 3s 13ms/step - loss: 1.1033 - categorical_accuracy: 0.6069 - val_loss: 1.6230 - val_categorical_accuracy: 0.4678
Epoch 486/500
196/196 [==============================] - 3s 13ms/step - loss: 1.0996 - categorical_accuracy: 0.6074 - val_loss: 1.8636 - val_categorical_accuracy: 0

Comparons les courbe de loss et d'accuracy de notre modèle principal avec nos deux modèles tests.

**Rappel** : En bleu foncé, les courbes du modèle principal sur les données de validation. En orange, les courbes du modèle principal sur les données d'entraînement.

En bleu clair, les courbes du modèle à 2 couches cachées sur les données de validation. En rouge, les courbes du modèle à 2 couches cachées sur les données d'entraînement.

En vert, les courbes du modèle à 20 couches cachées sur les données de validation. En rose, les courbes du modèle à 20 couches cachées sur les données d'entraînement.

![Comparaison_main_couches_test](images/MLP_a_accuracy.png)

![Comparaison_main_couches_test](images/MLP_a_loss.png)

Commençons par comparer avec le modèle à **2 couches cachées**, nous pouvons voir que sur les données de validation, ce modèle à presque les mêmes résultats que notre modèle principal. La majeur différence se fait sur les données d'entraînement où l'on peut voir que le modèle moins profond apprend moins bien que le modèle initial.

Pour le modèle à **20 couches cachées**, nous pouvons voir que l'apprentissage est plus lent de manière générale, cela est dû à la profondeur du modèle.

## Learning rate

Nous allons ajuster notre taux d'apprentissage (*learning rate* en anglais) de notre modèle, notre modèle initial contient un taux d'apprentissage de 0.01 (valeur par défaut). Nous allons comparer avec un modèle ayant 0.005 puis 0.05 en taux d'apprentissage pour voir l'importance de cette hyperparamètre sur notre modèle.

In [28]:
batch_size = 256
learning_rates = [0.0005, 0.05]
momentum = 0.45
num_layer = 6
nodes_by_layers = [32, 16, 64, 32, 16, 64]

In [31]:
for i, learning_rate in enumerate(learning_rates):
    mlp = MLP_main(num_layer, nodes_by_layers)
    
    mlp.compile(
        loss=categorical_crossentropy,
        optimizer=SGD(learning_rate=learning_rate, momentum=momentum),
        metrics=categorical_accuracy
    )
    
    MLP_LOG = os.path.join(LOG_DIR, "mlp",
                    f"ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{learning_rate}_mom_{momentum}_af_relu_layers_{'_'.join(str(e) for e in nodes_by_layers)}_b_{i+1}")
    
    MLP_MODELS = os.path.join(MODELS_DIR, "mlp",
                    f"ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{learning_rate}_mom_{momentum}_af_relu_layers_{'_'.join(str(e) for e in nodes_by_layers)}_b_{i+1}.keras")

    mlp.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[TensorBoard(MLP_LOG)]
           )
    
    mlp.save(MLP_MODELS)

2021-12-26 00:05:20.453373: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-26 00:05:20.453389: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-26 00:05:20.453411: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/500
 12/196 [>.............................] - ETA: 1s - loss: 2.4349 - categorical_accuracy: 0.0892

2021-12-26 00:05:21.235868: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-26 00:05:21.314886: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-26 00:05:21.314895: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-26 00:05:21.337736: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-26 00:05:21.338044: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-26 00:05:21.339008: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/mlp/ep_500_bs_256_opt_SGD_lr_0.0005_mom_0.45_af_relu_layers_32_16_64_32_16_64_b_1/train/plugins/profile/2021_12_26_00_05_21

2021-12-26 00:05:21.339407: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/mlp/ep_500_bs_25

196/196 [==============================] - ETA: 0s - loss: 2.3159 - categorical_accuracy: 0.0997

2021-12-26 00:05:22.958622: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

196/196 [==============================] - 2s 8ms/step - loss: 1.1487 - categorical_accuracy: 0.5848 - val_loss: 1.6399 - val_categorical_accuracy: 0.4617
Epoch 292/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1489 - categorical_accuracy: 0.5835 - val_loss: 1.6903 - val_categorical_accuracy: 0.4574
Epoch 293/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1468 - categorical_accuracy: 0.5847 - val_loss: 1.5945 - val_categorical_accuracy: 0.4621
Epoch 294/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1454 - categorical_accuracy: 0.5869 - val_loss: 1.5990 - val_categorical_accuracy: 0.4652
Epoch 295/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1490 - categorical_accuracy: 0.5842 - val_loss: 1.6552 - val_categorical_accuracy: 0.4493
Epoch 296/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1458 - categorical_accuracy: 0.5844 - val_loss: 1.7712 - val_categorical_accuracy: 0.4268


196/196 [==============================] - 2s 8ms/step - loss: 1.1258 - categorical_accuracy: 0.5935 - val_loss: 1.6239 - val_categorical_accuracy: 0.4627
Epoch 340/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1157 - categorical_accuracy: 0.5969 - val_loss: 1.6629 - val_categorical_accuracy: 0.4516
Epoch 341/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1198 - categorical_accuracy: 0.5956 - val_loss: 1.7271 - val_categorical_accuracy: 0.4519
Epoch 342/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1191 - categorical_accuracy: 0.5970 - val_loss: 1.7009 - val_categorical_accuracy: 0.4511
Epoch 343/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1228 - categorical_accuracy: 0.5934 - val_loss: 1.7368 - val_categorical_accuracy: 0.4462
Epoch 344/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1133 - categorical_accuracy: 0.5959 - val_loss: 1.6769 - val_categorical_accuracy: 0.4532


196/196 [==============================] - 2s 8ms/step - loss: 1.1003 - categorical_accuracy: 0.6022 - val_loss: 1.6429 - val_categorical_accuracy: 0.4609
Epoch 388/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0910 - categorical_accuracy: 0.6037 - val_loss: 1.6898 - val_categorical_accuracy: 0.4424
Epoch 389/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1044 - categorical_accuracy: 0.5992 - val_loss: 1.6967 - val_categorical_accuracy: 0.4487
Epoch 390/500
196/196 [==============================] - 2s 8ms/step - loss: 1.1005 - categorical_accuracy: 0.6021 - val_loss: 1.6596 - val_categorical_accuracy: 0.4642
Epoch 391/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0964 - categorical_accuracy: 0.6035 - val_loss: 1.7348 - val_categorical_accuracy: 0.4473
Epoch 392/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0923 - categorical_accuracy: 0.6030 - val_loss: 1.6578 - val_categorical_accuracy: 0.4598


196/196 [==============================] - 2s 8ms/step - loss: 1.0772 - categorical_accuracy: 0.6108 - val_loss: 1.6744 - val_categorical_accuracy: 0.4576
Epoch 436/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0743 - categorical_accuracy: 0.6098 - val_loss: 1.6919 - val_categorical_accuracy: 0.4642
Epoch 437/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0769 - categorical_accuracy: 0.6119 - val_loss: 1.8771 - val_categorical_accuracy: 0.4311
Epoch 438/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0842 - categorical_accuracy: 0.6085 - val_loss: 1.6932 - val_categorical_accuracy: 0.4532
Epoch 439/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0821 - categorical_accuracy: 0.6070 - val_loss: 1.7363 - val_categorical_accuracy: 0.4480
Epoch 440/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0744 - categorical_accuracy: 0.6112 - val_loss: 1.7218 - val_categorical_accuracy: 0.4540


196/196 [==============================] - 2s 8ms/step - loss: 1.0629 - categorical_accuracy: 0.6135 - val_loss: 1.7320 - val_categorical_accuracy: 0.4567
Epoch 484/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0595 - categorical_accuracy: 0.6142 - val_loss: 1.9902 - val_categorical_accuracy: 0.4291
Epoch 485/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0539 - categorical_accuracy: 0.6200 - val_loss: 1.7198 - val_categorical_accuracy: 0.4525
Epoch 486/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0535 - categorical_accuracy: 0.6176 - val_loss: 1.9083 - val_categorical_accuracy: 0.4354
Epoch 487/500
196/196 [==============================] - 2s 8ms/step - loss: 1.0619 - categorical_accuracy: 0.6174 - val_loss: 1.7263 - val_categorical_accuracy: 0.4504
Epoch 488/500
196/196 [==============================] - 2s 9ms/step - loss: 1.0556 - categorical_accuracy: 0.6165 - val_loss: 1.9172 - val_categorical_accuracy: 0.4223


Comparons les courbe de loss et d'accuracy de notre modèle principal avec nos deux modèles tests.

**Rappel** : En bleu foncé, les courbes du modèle principal sur les données de validation. 

En orange, les courbes du modèle principal sur les données d'entraînement.

En bleu clair, les courbes du modèle ayant un taux d'apprentissage de 0.005 sur les données de validation. 

En rouge, les courbes du modèle ayant un taux d'apprentissage de 0.005 sur les données d'entraînement.

En vert, les courbes du modèle ayant un taux d'apprentissage de 0.5 sur les données de validation. 

En rose, les courbes du modèle ayant un taux d'apprentissage de 0.5 sur les données d'entraînement.

![Comparaison_main_learning_rate_test](images/MLP_b_accuracy.png)

![Comparaison_main_learning_rate_test](images/MLP_b_loss.png)

Commençons par comparer avec le modèle ayant un **taux d'apprentissage de 0.005**, nous pouvons voir que ce modèle est de manière générale plus stable mais apprend plus lentement, ce qui est tout à fait normal car notre modèle réalise de tout petit pas vers le minimum local (si, on a de la chance vers minimum global).

Pour le modèle ayant un **taux d'apprentissage de 0.05**, nous pouvons voir que ce modèle à un apprentissage plus instable que le modèle initial, cela est dû au fort taux d'apprentissage qui réalise des pas très grand et diverge du minimum local.

## Batch size

Nous allons changer le nombre de couches de notre modèle, notre modèle initial contient un batch size de 256. Nous allons comparer avec un modèle contenant un batch size de 32 puis de 1024 pour voir l'importance de cette hyperparamètre.

In [33]:
batch_sizes = [32, 1024]
learning_rate = 0.01
momentum = 0.45
num_layers = 6
nodes_by_layers = [32, 16, 64, 32, 16, 64]

In [34]:
for i, batch_size in enumerate(batch_sizes):
    mlp = MLP_main(num_layers, nodes_by_layers)

    mlp.compile(
        loss=categorical_crossentropy,
        optimizer=SGD(learning_rate=learning_rate, momentum=momentum),
        metrics=categorical_accuracy
               )

    MLP_LOG = os.path.join(LOG_DIR, "mlp",
                    f"ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{learning_rate}_mom_{momentum}_af_relu_layers_{'_'.join(str(e) for e in nodes_by_layers)}_c_{i+1}")

    MLP_MODELS = os.path.join(MODELS_DIR, "mlp",
                    f"ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{learning_rate}_mom_{momentum}_af_relu_layers_{'_'.join(str(e) for e in nodes_by_layers)}_c_{i+1}.keras")

    mlp.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[TensorBoard(MLP_LOG)]
           )
    mlp.save(MLP_MODELS)

2021-12-26 00:59:31.909154: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-26 00:59:31.909169: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-26 00:59:31.909282: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/500
  12/1563 [..............................] - ETA: 15s - loss: 2.3319 - categorical_accuracy: 0.1146

2021-12-26 00:59:32.851876: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-26 00:59:32.925045: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-26 00:59:32.925053: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-26 00:59:32.948028: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-26 00:59:32.948337: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-26 00:59:32.949401: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/mlp/ep_500_bs_32_opt_SGD_lr_0.01_mom_0.45_af_relu_layers_32_16_64_32_16_64_c_1/train/plugins/profile/2021_12_26_00_59_32

2021-12-26 00:59:32.949919: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/mlp/ep_500_bs_32_op

1563/1563 [==============================] - ETA: 0s - loss: 2.0454 - categorical_accuracy: 0.2265

2021-12-26 00:59:43.996558: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

Epoch 296/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2922 - categorical_accuracy: 0.5365 - val_loss: 1.4754 - val_categorical_accuracy: 0.4796
Epoch 297/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2814 - categorical_accuracy: 0.5404 - val_loss: 1.4996 - val_categorical_accuracy: 0.4693
Epoch 298/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2758 - categorical_accuracy: 0.5407 - val_loss: 1.4946 - val_categorical_accuracy: 0.4712
Epoch 299/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2805 - categorical_accuracy: 0.5397 - val_loss: 1.4680 - val_categorical_accuracy: 0.4840
Epoch 300/500
49/49 [==============================] - 1s 11ms/step - loss: 1.2752 - categorical_accuracy: 0.5431 - val_loss: 1.7108 - val_categorical_accuracy: 0.4245
Epoch 301/500
49/49 [==============================] - 1s 10ms/step - loss: 1.3023 - categorical_accuracy: 0.5358 - val_loss: 1.4708 - val_categorical_accuracy:

Epoch 345/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2561 - categorical_accuracy: 0.5512 - val_loss: 1.5091 - val_categorical_accuracy: 0.4679
Epoch 346/500
49/49 [==============================] - 1s 11ms/step - loss: 1.2604 - categorical_accuracy: 0.5493 - val_loss: 1.4804 - val_categorical_accuracy: 0.4785
Epoch 347/500
49/49 [==============================] - 1s 11ms/step - loss: 1.2546 - categorical_accuracy: 0.5478 - val_loss: 1.5481 - val_categorical_accuracy: 0.4684
Epoch 348/500
49/49 [==============================] - 1s 11ms/step - loss: 1.2662 - categorical_accuracy: 0.5463 - val_loss: 1.5010 - val_categorical_accuracy: 0.4709
Epoch 349/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2492 - categorical_accuracy: 0.5528 - val_loss: 1.5069 - val_categorical_accuracy: 0.4734
Epoch 350/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2756 - categorical_accuracy: 0.5421 - val_loss: 1.4713 - val_categorical_accuracy:

Epoch 394/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2373 - categorical_accuracy: 0.5559 - val_loss: 1.5388 - val_categorical_accuracy: 0.4680
Epoch 395/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2413 - categorical_accuracy: 0.5570 - val_loss: 1.5185 - val_categorical_accuracy: 0.4663
Epoch 396/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2416 - categorical_accuracy: 0.5542 - val_loss: 1.4933 - val_categorical_accuracy: 0.4756
Epoch 397/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2285 - categorical_accuracy: 0.5614 - val_loss: 1.4994 - val_categorical_accuracy: 0.4788
Epoch 398/500
49/49 [==============================] - 1s 11ms/step - loss: 1.2308 - categorical_accuracy: 0.5593 - val_loss: 1.4731 - val_categorical_accuracy: 0.4847
Epoch 399/500
49/49 [==============================] - 1s 11ms/step - loss: 1.2433 - categorical_accuracy: 0.5544 - val_loss: 1.4931 - val_categorical_accuracy:

Epoch 443/500
49/49 [==============================] - 1s 11ms/step - loss: 1.2200 - categorical_accuracy: 0.5633 - val_loss: 1.4789 - val_categorical_accuracy: 0.4828
Epoch 444/500
49/49 [==============================] - 1s 11ms/step - loss: 1.2228 - categorical_accuracy: 0.5634 - val_loss: 1.4889 - val_categorical_accuracy: 0.4807
Epoch 445/500
49/49 [==============================] - 1s 11ms/step - loss: 1.2119 - categorical_accuracy: 0.5691 - val_loss: 1.4785 - val_categorical_accuracy: 0.4805
Epoch 446/500
49/49 [==============================] - 1s 11ms/step - loss: 1.2251 - categorical_accuracy: 0.5616 - val_loss: 1.5305 - val_categorical_accuracy: 0.4698
Epoch 447/500
49/49 [==============================] - 1s 11ms/step - loss: 1.1952 - categorical_accuracy: 0.5716 - val_loss: 1.4895 - val_categorical_accuracy: 0.4806
Epoch 448/500
49/49 [==============================] - 1s 11ms/step - loss: 1.2242 - categorical_accuracy: 0.5650 - val_loss: 1.6255 - val_categorical_accuracy:

Epoch 492/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2114 - categorical_accuracy: 0.5667 - val_loss: 1.5360 - val_categorical_accuracy: 0.4682
Epoch 493/500
49/49 [==============================] - 1s 11ms/step - loss: 1.1969 - categorical_accuracy: 0.5729 - val_loss: 1.4900 - val_categorical_accuracy: 0.4788
Epoch 494/500
49/49 [==============================] - 1s 10ms/step - loss: 1.2200 - categorical_accuracy: 0.5635 - val_loss: 1.5883 - val_categorical_accuracy: 0.4594
Epoch 495/500
49/49 [==============================] - 1s 11ms/step - loss: 1.2043 - categorical_accuracy: 0.5692 - val_loss: 1.5218 - val_categorical_accuracy: 0.4761
Epoch 496/500
49/49 [==============================] - 1s 11ms/step - loss: 1.1831 - categorical_accuracy: 0.5746 - val_loss: 1.5346 - val_categorical_accuracy: 0.4684
Epoch 497/500
49/49 [==============================] - 1s 11ms/step - loss: 1.1894 - categorical_accuracy: 0.5756 - val_loss: 1.5440 - val_categorical_accuracy:

Comparons les courbe de loss et d'accuracy de notre modèle principal avec nos deux modèles tests.

**Rappel** : En bleu foncé, les courbes du modèle principal sur les données de validation. 

En orange, les courbes du modèle principal sur les données d'entraînement.

En bleu clair, les courbes du modèle ayant un batch size de 32 sur les données de validation.

En rouge, les courbes du modèle ayant un batch size de 32 sur les données d'entraînement.

En vert, les courbes du modèle ayant un batch size de 1024 sur les données de validation.

En rose, les courbes du modèle ayant un batch size de 1024 sur les données d'entraînement.

![Comparaison_main_batch_test](images/MLP_c_accuracy.png)

![Comparaison_main_batch_test](images/MLP_c_loss.png)

Commençons par comparer avec le modèle ayant un **batch size de 32**, nous pouvons voir que l'apprentissage est plus lent de manière générale, cela est dû à la profondeur du modèle. Mais pourquoi ?

Pour le modèle ayant un **batch size de 1024**, ??

## Fonction d'activations

----

---

In [10]:
learning_rates = [0.02, 0.07, 0.1]
momentums = [0.25, 0.5, 0.9]

for lr in learning_rates:
    for mom in momentums:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        nodes_by_layers = np.random.choice(NUM_UNITS, num_layer)
        mlp = MLP(num_layer, nodes_by_layers)
        mlp.compile(loss=categorical_crossentropy,
                     optimizer=SGD(learning_rate=lr,
                                   momentum=mom),
                     metrics=categorical_accuracy)
        MLP_LOG = os.path.join(LOG_DIR, "mlp",
                        f"mlp_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in nodes_by_layers)}")

        mlp.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(MLP_LOG, histogram_freq=1)]
               )

2021-12-24 00:25:45.458424: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-24 00:25:45.458445: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-24 00:25:45.458474: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/200
 1/98 [..............................] - ETA: 17s - loss: 2.4496 - categorical_accuracy: 0.0723

2021-12-24 00:25:46.399287: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-24 00:25:46.402058: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-24 00:25:46.515000: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


21/98 [=====>........................] - ETA: 0s - loss: 2.2988 - categorical_accuracy: 0.1140

2021-12-24 00:25:46.602177: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-24 00:25:46.602185: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-24 00:25:46.612179: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-24 00:25:46.614994: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-24 00:25:46.619435: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/mlp/mlp_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_16_32/train/plugins/profile/2021_12_24_00_25_46

2021-12-24 00:25:46.620264: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/mlp/mlp_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_16_32/train/plugins/profile/2021_12_24_00_25_46/MacBook-Pro-de-Reda.local.trace.json.gz
2021-12-24 00:25:46.623171: I tensorflow/core/profi

...

2021-12-24 00:34:33.198959: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-24 00:34:33.274815: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-24 00:34:33.274824: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-24 00:34:33.294855: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-24 00:34:33.295134: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-24 00:34:33.296111: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/mlp/mlp_ep_200_bs_512_opt_SGD_lr_0.07_mom_0.25_layers_64_128_8_64_16/train/plugins/profile/2021_12_24_00_34_33

2021-12-24 00:34:33.296509: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/mlp/mlp_ep_200_bs_512_opt_SGD

98/98 [==============================] - 1s 11ms/step - loss: 2.2831 - categorical_accuracy: 0.1326 - val_loss: 2.3041 - val_categorical_accuracy: 0.1007
Epoch 2/200
 1/98 [..............................] - ETA: 0s - loss: 2.3057 - categorical_accuracy: 0.0859

2021-12-24 00:34:34.211081: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


98/98 [==============================] - 1s 9ms/step - loss: 2.2321 - categorical_accuracy: 0.1753 - val_loss: 2.1095 - val_categorical_accuracy: 0.2548
Epoch 3/200
98/98 [==============================] - 1s 9ms/step - loss: 2.0919 - categorical_accuracy: 0.2374 - val_loss: 2.1977 - val_categorical_accuracy: 0.1857
Epoch 4/200
98/98 [==============================] - 1s 9ms/step - loss: 2.0196 - categorical_accuracy: 0.2658 - val_loss: 1.9599 - val_categorical_accuracy: 0.2935
Epoch 5/200
98/98 [==============================] - 1s 9ms/step - loss: 1.9535 - categorical_accuracy: 0.2911 - val_loss: 1.9618 - val_categorical_accuracy: 0.2643
Epoch 6/200
98/98 [==============================] - 1s 9ms/step - loss: 1.9218 - categorical_accuracy: 0.2978 - val_loss: 1.8944 - val_categorical_accuracy: 0.3160
Epoch 7/200
98/98 [==============================] - 1s 9ms/step - loss: 1.8994 - categorical_accuracy: 0.3077 - val_loss: 1.8998 - val_categorical_accuracy: 0.3133
Epoch 8/200
98/98 [===

KeyboardInterrupt: 

In [ ]:
batch_size = 256
lr = 0.01
mom = 0.45
num_layer = 6
nodes_by_layers = [32, 16, 64, 32, 16, 64]


mlp = MLP(num_layer, nodes_by_layers)
mlp.compile(loss=categorical_crossentropy,
             optimizer=SGD(learning_rate=lr,
                           momentum=mom),
             metrics=categorical_accuracy)
MLP_LOG = os.path.join(LOG_DIR, "mlp",
                f"ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{lr}_mom_{mom}_af_relu_layers_{'_'.join(str(e) for e in nodes_by_layers)}_main")
MLP_MODELS = os.path.join(MODELS_DIR, "mlp",
                f"ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{lr}_mom_{mom}_af_relu_layers_{'_'.join(str(e) for e in nodes_by_layers)}_main.keras")

mlp.fit(x_train,
        y_train,
        batch_size=batch_size,
        epochs=EPOCHS,
        validation_data=(x_test, y_test),
        shuffle=SHUFFLE,
        callbacks=[TensorBoard(MLP_LOG, histogram_freq=1)]
       )
mlp.save(MLP_MODELS)

# ConvNet

In [21]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True

In [22]:
learning_rates = [0.02, 0.07, 0.1]
momentums = [0.25, 0.5, 0.9]


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=tanh)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)

for lr in learning_rates:
    for mom in momentums:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32*x for x in range(1, num_layer+1)]

        convnet = ConvNet(num_layer, filters_by_layers)
        convnet.compile(loss=categorical_crossentropy,
                    optimizer=SGD(lr, momentum=mom),
                    metrics=categorical_accuracy)
        CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
        convnet.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_test, y_test),
                    shuffle=SHUFFLE,
                    callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)]
                   )
                       

2021-12-20 20:33:21.053470: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-20 20:33:21.053493: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-20 20:33:21.053634: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/200


2021-12-20 20:33:21.989900: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


 2/98 [..............................] - ETA: 10s - loss: 2.3092 - categorical_accuracy: 0.0918

2021-12-20 20:33:22.270257: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-20 20:33:22.270268: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-20 20:33:22.386319: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-20 20:33:22.386702: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-20 20:33:22.387393: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/convnet/convnet_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_32_64_96_128_160_af_tanh/train/plugins/profile/2021_12_20_20_33_22

2021-12-20 20:33:22.387897: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/convnet/convnet_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_32_64_96_128_160_af_tanh/train/plugins/profile/2021_12_20_20_33_22/MacBook-Pro-de-Reda.local.trace.json.

98/98 [==============================] - ETA: 0s - loss: 2.2679 - categorical_accuracy: 0.1576

2021-12-20 20:33:30.972801: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


98/98 [==============================] - 10s 95ms/step - loss: 2.2679 - categorical_accuracy: 0.1576 - val_loss: 2.2226 - val_categorical_accuracy: 0.2287
Epoch 2/200
98/98 [==============================] - 8s 85ms/step - loss: 2.1245 - categorical_accuracy: 0.2772 - val_loss: 2.0017 - val_categorical_accuracy: 0.3101
Epoch 3/200
98/98 [==============================] - 8s 84ms/step - loss: 1.9581 - categorical_accuracy: 0.3084 - val_loss: 1.9868 - val_categorical_accuracy: 0.2860
Epoch 4/200
98/98 [==============================] - 8s 86ms/step - loss: 1.8658 - categorical_accuracy: 0.3375 - val_loss: 1.7984 - val_categorical_accuracy: 0.3671
Epoch 5/200
98/98 [==============================] - 8s 83ms/step - loss: 1.8069 - categorical_accuracy: 0.3546 - val_loss: 1.7358 - val_categorical_accuracy: 0.3881
Epoch 6/200
42/98 [===========>..................] - ETA: 4s - loss: 1.7631 - categorical_accuracy: 0.3711

KeyboardInterrupt: 

# ResNet

In [116]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True

In [121]:
learning_rates = [0.02, 0.07, 0.1]
momentums = [0.25, 0.5, 0.9]


def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=relu,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)

for lr in learning_rates:
    for mom in momentums:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32 for x in range(1, num_layer+1)]

        resnet = ResNet(num_layer, filters_by_layers)
        resnet.compile(loss=categorical_crossentropy,
                    optimizer=SGD(lr, momentum=mom),
                    metrics=categorical_accuracy)
        RESNET_LOG = os.path.join(LOG_DIR, "resnet",
                        f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu")
        resnet.fit(x_train,
                        y_train,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(x_test, y_test),
                        shuffle=SHUFFLE,
                        callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
                       )
                       

2021-12-19 17:01:57.606703: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-19 17:01:57.606856: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-19 17:01:57.607374: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
/Users/redamaizate/miniforge3/envs/cours_env/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
2021-12-19 17:01:58.789158: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/200
 1/98 [..............................] - ETA: 2:20 - loss: 2.2979 - categorical_accuracy: 0.1289

2021-12-19 17:02:00.092889: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-19 17:02:00.092900: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


 2/98 [..............................] - ETA: 23s - loss: 2.3021 - categorical_accuracy: 0.1279 

2021-12-19 17:02:00.353738: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-19 17:02:00.357145: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-19 17:02:00.361354: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/resnet/resnet_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_32_32_32_32_32_af_relu/train/plugins/profile/2021_12_19_17_02_00

2021-12-19 17:02:00.362273: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/resnet/resnet_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_32_32_32_32_32_af_relu/train/plugins/profile/2021_12_19_17_02_00/MacBook-Pro-de-Reda.local.trace.json.gz
2021-12-19 17:02:00.366658: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/resnet/resnet_ep_200_bs_512_opt_SGD_lr_0.02_mom_0.25_layers_32_32_32_32_32_af_relu/train/plugins/profile/2021_12_19_17

98/98 [==============================] - ETA: 0s - loss: 2.1314 - categorical_accuracy: 0.2270

2021-12-19 17:02:18.908965: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


98/98 [==============================] - 22s 212ms/step - loss: 2.1314 - categorical_accuracy: 0.2270 - val_loss: 2.0337 - val_categorical_accuracy: 0.2997
Epoch 2/200
98/98 [==============================] - 20s 206ms/step - loss: 1.9130 - categorical_accuracy: 0.3304 - val_loss: 1.8485 - val_categorical_accuracy: 0.3420
Epoch 3/200
98/98 [==============================] - 21s 211ms/step - loss: 1.7972 - categorical_accuracy: 0.3718 - val_loss: 1.9520 - val_categorical_accuracy: 0.3156
Epoch 4/200
98/98 [==============================] - 20s 204ms/step - loss: 1.7322 - categorical_accuracy: 0.3963 - val_loss: 1.6697 - val_categorical_accuracy: 0.4106
Epoch 5/200
98/98 [==============================] - 20s 205ms/step - loss: 1.6770 - categorical_accuracy: 0.4139 - val_loss: 1.6319 - val_categorical_accuracy: 0.4241
Epoch 6/200
98/98 [==============================] - 20s 200ms/step - loss: 1.6437 - categorical_accuracy: 0.4246 - val_loss: 1.6178 - val_categorical_accuracy: 0.4277
Epoc

KeyboardInterrupt: 

# RNN